In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import f1_score, accuracy_score, precision_score, mean_squared_error, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
# Data Augmentation and Normalization
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
full_dataset = torchvision.datasets.ImageFolder(root='D:/capstone/Train Dataset', transform=transform_train)
train_size = int(0.9 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)

test_dataset = torchvision.datasets.ImageFolder(root='D:/capstone/Test Dataset', transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

num_classes = len(full_dataset.classes)

In [6]:
def extract_features(model, dataloader):
    features = []
    labels = []
    model.eval()
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            features.append(outputs.cpu().numpy())
            labels.append(targets.numpy())
    return np.concatenate(features), np.concatenate(labels)

In [7]:
def fine_tune_model(model, num_classes):
    if isinstance(model, torchvision.models.EfficientNet):
        for param in model.features.parameters():
            param.requires_grad = True
        num_ftrs = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(num_ftrs, 1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
    elif isinstance(model, torchvision.models.VisionTransformer):
        for param in model.encoder.layers[-6:].parameters():
            param.requires_grad = True
        num_ftrs = model.heads.head.in_features
        model.heads.head = nn.Sequential(
            nn.Linear(num_ftrs, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )
    elif isinstance(model, torchvision.models.DenseNet):
        for param in model.features.denseblock4.parameters():
            param.requires_grad = True
        num_ftrs = model.classifier.in_features
        model.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(num_ftrs, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )
    
    return model

In [8]:
# Initialize models
efficientnet = fine_tune_model(torchvision.models.efficientnet_v2_s(pretrained=False), num_classes).to(device)
vision_transformer = fine_tune_model(torchvision.models.vit_b_16(pretrained=False), num_classes).to(device)
densenet = fine_tune_model(torchvision.models.densenet201(pretrained=False), num_classes).to(device)

C:\Users\mridu\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\mridu\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [15]:
def load_model_weights(model, weight_path):
    model.load_state_dict(torch.load(weight_path,map_location ='cpu'))
    model.eval()
    return model

In [17]:
efficientnet = load_model_weights(efficientnet,'EfficientNet_best.pth')

C:\Users\mridu\AppData\Local\Temp\ipykernel_33648\1876342858.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weight_path,map_location ='

In [19]:
vision_transformer = load_model_weights(vision_transformer,'VisionTransformer_best.pth')

C:\Users\mridu\AppData\Local\Temp\ipykernel_33648\1876342858.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weight_path,map_location ='

In [21]:
densenet = load_model_weights(densenet,'DenseNet_best.pth')

C:\Users\mridu\AppData\Local\Temp\ipykernel_33648\1876342858.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weight_path,map_location ='

In [23]:
models = {
    'efficientnet': efficientnet,
    'vision_transformer': vision_transformer,
    'densenet': densenet
}

In [25]:
# Extract features
def extract_features_all_models(models, dataloader):
    all_features = []
    labels = None
    for name, model in models.items():
        features, labels = extract_features(model, dataloader)
        all_features.append(features)
    return np.concatenate(all_features, axis=1), labels


In [27]:
train_features, train_labels = extract_features_all_models(models, train_loader)
val_features, val_labels = extract_features_all_models(models, val_loader)
test_features, test_labels = extract_features_all_models(models, test_loader)


In [ ]:
# def changes(X):
#     X_changed = X.copy()  # Create a copy to avoid modifying the original dataset
#     for i in range(5):
#         value = (X[i] + X[i + 5] + X[i + 10]) / 3  # Average the values
#         X_changed[i] = value  # Replace the value at index i
#     return X_changed


In [ ]:
# # Apply changes to train, val, and test features
# train_features_changed = changes(train_features)
# val_features_changed = changes(val_features)
# test_features_changed = changes(test_features)

In [ ]:
# # scaler = StandardScaler()
# # train_features_normalized = scaler.fit_transform(train_features)
# # val_features_normalized = scaler.transform(val_features)
# # test_features_normalized = scaler.transform(test_features)
# train_features_changed.shape

In [29]:
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
val_features_normalized = scaler.transform(val_features)
test_features_normalized = scaler.transform(test_features)

In [31]:
def create_random_forest_ensemble(X_train, y_train, X_val, y_val):
    rf_classifier = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, n_jobs=-1)
    rf_classifier.fit(X_train, y_train)
    
    val_preds = rf_classifier.predict(X_val)
    val_accuracy = accuracy_score(y_val, val_preds)
    print(f"Random Forest Ensemble Validation Accuracy: {val_accuracy:.4f}")
    
    return rf_classifier

In [33]:
# Create and evaluate Random Forest ensemble
rf_ensemble = create_random_forest_ensemble(train_features_normalized, train_labels, val_features_normalized, val_labels)



Random Forest Ensemble Validation Accuracy: 0.9655


In [35]:
# Make predictions on test set
test_preds = rf_ensemble.predict(test_features_normalized)
test_accuracy = accuracy_score(test_labels, test_preds)
print(f"Random Forest Ensemble Test Accuracy: {test_accuracy:.4f}")

Random Forest Ensemble Test Accuracy: 0.9781


In [37]:
# Calculate metrics
def calculate_metrics(y_true, y_pred):
    return {
        'f1_score': f1_score(y_true, y_pred, average='weighted'),
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted'),
        'mse': mean_squared_error(y_true, y_pred)
    }


In [39]:
train_preds = rf_ensemble.predict(train_features_normalized)
val_preds = rf_ensemble.predict(val_features_normalized)

train_metrics = calculate_metrics(train_labels, train_preds)
val_metrics = calculate_metrics(val_labels, val_preds)
test_metrics = calculate_metrics(test_labels, test_preds)

In [41]:
print("Train metrics:", train_metrics)
print("Validation metrics:", val_metrics)
print("Test metrics:", test_metrics)

Train metrics: {'f1_score': 0.9871344484316356, 'accuracy': 0.9871232876712329, 'precision': 0.9871611151152891, 'mse': 0.02493150684931507}
Validation metrics: {'f1_score': 0.9654782007169014, 'accuracy': 0.9655172413793104, 'precision': 0.9655213416457795, 'mse': 0.09852216748768473}
Test metrics: {'f1_score': 0.9780899421027046, 'accuracy': 0.9781021897810219, 'precision': 0.9783750054956519, 'mse': 0.049009384775808136}
